In [1]:
import sys, os, time, json, pickle
from timeit import default_timer as timer
from humanfriendly import format_timespan
from glob import glob

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
%matplotlib inline

In [4]:
from sklearn.externals import joblib

In [5]:
from utils.autoreview_utils import get_best_model_from_datadir, prepare_data_for_model, predict_ranks_from_data

In [6]:
from dotenv import load_dotenv
load_dotenv('admin.env')

True

In [7]:
from mysql_connect import get_db_connection
db = get_db_connection('jp_autoreview')

In [8]:
start = timer()
df = db.read_sql(db.tables['pipeline_tests'].select())
print(format_timespan(timer()-start))

1.77 second


In [9]:
df.set_index('id', inplace=True)

In [10]:
top_results = df.sort_values('score_correctly_predicted', ascending=False).drop_duplicates(['review_paper_id', 'random_seed'])

In [11]:
top_results = top_results.sort_values('num_target_papers', ascending=False)

In [18]:
# r = top_results.sample(n=1, random_state=999)
r = top_results.sample(n=1)
r = r.iloc[0]
datadir = r.datadir

In [19]:
pipeline = get_best_model_from_datadir(datadir)

In [20]:
test_papers, seed_papers, train_papers = prepare_data_for_model(datadir)

In [21]:
predictions = predict_ranks_from_data(pipeline, test_papers)

In [23]:
predictions.head(20)

,EF,Paper_ID,cl,title,year,target,pred_ranks
1369,7.617600e-06,1980216394,129366:1:7:1:1,a low cost high efficiency solar cell based on...,1991.0,True,0.671015
1794,5.224840e-07,1991661990,8254:13:1,formation of titanium oxide nanotube,1998.0,True,0.664111
2802,1.376610e-05,2018858346,37791:10:1:1,helical microtubules of graphitic carbon,1991.0,True,0.628000
5967,2.443290e-06,2121304740,129366:1:13:1,conversion of light to electricity by cis x2bi...,1993.0,True,0.594432
435486,3.463660e-07,2123025816,8254:13:2,titania nanotubes prepared by chemical processing,1999.0,True,0.582453
5797,2.344510e-06,2111062994,8254:22:1,visible light photocatalysis in nitrogen doped...,2001.0,True,0.580896
7807,2.048980e-06,2494630477,129366:1:12:1,photoelectrochemical cells materials for clean...,2001.0,True,0.553081
22421,9.511190e-08,2032162308,2028213:1:19,dye sensitized anodic tio2 nanotubes,2005.0,True,0.550721
6530,1.559510e-06,2157254946,940207:1:3:1,ordered metal nanohole arrays made by a two st...,1995.0,True,0.541112
111229,9.158110e-09,2006102206,2028213:1:219,water annealing and other low temperature trea...,2012.0,True,0.535755


In [24]:
new_r = top_results.sample(n=1)
new_r = new_r.iloc[0]
new_datadir = new_r['datadir']

In [25]:
test_papers, seed_papers, train_papers = prepare_data_for_model(new_datadir)

In [26]:
new_predictions = predict_ranks_from_data(pipeline, test_papers)

In [27]:
new_predictions.head(20)

,EF,Paper_ID,cl,title,year,target,pred_ranks
241758,7.617600e-06,1980216394,129366:1:7:1:1,a low cost high efficiency solar cell based on...,1991.0,False,0.576603
1398715,2.048980e-06,2494630477,129366:1:12:1,photoelectrochemical cells materials for clean...,2001.0,False,0.501053
1122196,1.072070e-05,1994869187,37791:13:1:1,science of fullerenes and carbon nanotubes,1997.0,False,0.444443
1416253,5.254490e-07,2129779013,940207:1:1:1,the morphology and mechanism of formation of p...,1970.0,False,0.303113
424800,5.892610e-06,2077106134,620639:1:2,dynamics of fluids in porous media,1975.0,False,0.267518
1783659,6.335630e-06,2042587503,434855:2,a measure of asymptotic efficiency for tests o...,1952.0,False,0.198071
1001643,7.802630e-07,1973613246,129366:1:39:1,nanocrystalline titanium oxide electrodes for ...,2005.0,False,0.197481
655939,2.349420e-06,2119585240,8254:6:2,photocatalysis on tio2 surfaces principles mec...,1995.0,False,0.135625
613703,6.331490e-06,2039810561,41735:2:1:3,a theory of the allocation of time,1965.0,False,0.130071
1527205,1.373220e-05,2159401492,32409:2:1:1,applied multiple regression correlation analys...,1975.0,False,0.104000


In [30]:
def try_random_data(pipeline):
    r = top_results.sample(n=1)
    r = r.iloc[0]
    datadir = r['datadir']
    test_papers, seed_papers, train_papers = prepare_data_for_model(datadir)
    predictions = predict_ranks_from_data(pipeline, test_papers)
    return r.name, predictions

In [33]:
name, preds = try_random_data(pipeline)
name

24859

In [34]:
preds.head(20)

,EF,Paper_ID,cl,title,year,target,pred_ranks
868278,5.955870e-06,1562208008,353056:21:1:1,self efficacy the exercise of control,1997.0,False,0.291518
335401,8.183950e-09,2129327132,176693:10:96,noradrenaline infused locally reduces arrhythm...,2002.0,False,0.078560
169521,9.777200e-09,1982178254,835503:3:129,brain repair an overview,1994.0,False,0.078560
305275,7.417000e-09,2120889461,102359:8:25,glutamate activates pp125 fak through ampa kai...,2001.0,False,0.078560
692561,1.511110e-08,2048343908,3647816:7:49,oxidized ldl containing immune complexes induc...,1999.0,False,0.078560
755069,5.762140e-08,2117696416,3585070:7:1,targeting antioxidants to mitochondria a new t...,2006.0,False,0.078560
52734,1.084170e-07,2053203632,1408075:7:3,carnitine deficiency presenting as familial ca...,1982.0,False,0.078560
688187,5.833930e-09,1532450218,2651778:1:466,genetic approach to biosequence alignment gaba,2002.0,False,0.078560
350372,2.730190e-08,1999346088,1073019:7:148,chronic intermittent hypoxia predisposes to li...,2007.0,False,0.078560
89705,2.726710e-08,1986191342,95381:3:39,nfat transcription factors are critical surviv...,2003.0,False,0.078560


In [37]:
name, preds = try_random_data(pipeline)
name

21067

In [38]:
preds.head(20)

,EF,Paper_ID,cl,title,year,target,pred_ranks
199691,5.224840e-07,1991661990,8254:13:1,formation of titanium oxide nanotube,1998.0,False,0.756904
171060,1.376610e-05,2018858346,37791:10:1:1,helical microtubules of graphitic carbon,1991.0,True,0.562000
167723,7.489460e-06,1560640234,37791:15:1,physical properties of carbon nanotubes,1998.0,False,0.418501
17976,3.040780e-06,1646294624,37791:2:3,room temperature transistor based on a single ...,1998.0,False,0.350302
130614,4.643950e-06,2008302913,37791:2:1,crystalline ropes of metallic carbon nanotubes,1996.0,False,0.334139
214927,2.678520e-06,2094963548,37791:30:1,carbon nanotubes the route toward applications,2002.0,False,0.280713
211333,5.902080e-06,2023673413,571294:1:1,solar energy materials,2003.0,False,0.271518
194019,5.892610e-06,2077106134,620639:1:2,dynamics of fluids in porous media,1975.0,False,0.267518
214329,3.287440e-06,2082904789,37791:2:2,single shell carbon nanotubes of 1 nm diameter,1993.0,False,0.264289
232248,6.335630e-06,2042587503,434855:2,a measure of asymptotic efficiency for tests o...,1952.0,False,0.256071


In [41]:
r.clf

"RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',\n            max_depth=None, max_features='auto', max_leaf_nodes=None,\n            min_impurity_decrease=0.0, min_impurity_split=None,\n            min_samples_leaf=1, min_samples_split=2,\n            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,\n            oob_score=False, random_state=999, verbose=0, warm_start=False)"

In [43]:
str(pipeline._final_estimator) == r.clf

True

In [45]:
start = timer()
matches = {}
for row_idx, row in top_results.iterrows():
    pipeline = get_best_model_from_datadir(row.datadir)
    matches[row_idx] = (str(pipeline._final_estimator) == row.clf)
print(format_timespan(timer()-start))

KeyboardInterrupt: 

In [49]:
from sqlalchemy.orm import sessionmaker
from models.jp_autoreview import PipelineTest

In [50]:
Session = sessionmaker(bind=db.engine)

In [51]:
session = Session()

In [59]:
start = timer()
record = session.query(PipelineTest).filter_by(datadir=row.datadir).filter_by(clf=str(pipeline._final_estimator)).all()
print(format_timespan(timer()-start))

0.08 seconds


In [65]:
record[1].id

2697

In [76]:
def get_feature_names(pipeline):
    u = pipeline.named_steps['union']
    features = []
    for item in u.transformer_list:
        features.append(item[0])
    return features

In [77]:
get_feature_names(pipeline)

['avg_distance_to_train', 'ef']

In [73]:
u = pipeline.named_steps['union']
u.transformer_list[1][0]

'ef'

In [82]:
for item in record:
    print(item.features == str(get_feature_names(pipeline)))
item.features == str(get_feature_names(pipeline))

True
False


False

In [87]:
all_datadirs = df.datadir.unique().tolist()

In [100]:
start = timer()
i = 0
num_models = 0
collect_data = []
for datadir in all_datadirs:
    g = glob(os.path.join(datadir, 'best_model*'))
    for model_path in g:
        fname = os.path.join(model_path, 'best_model.pickle')
        if os.path.exists(fname):
            num_models += 1
#         pipeline = joblib.load(fname)
#         features = str(get_feature_names(pipeline))
#         r = session.query(PipelineTest) \
#                     .filter_by(datadir=datadir) \
#                     .filter_by(clf=str(pipeline._final_estimator)) \
#                     .filter_by(features=features) \
#                     .all()
#         collect_data.append(r)
    i += 1

print(format_timespan(timer()-start))

KeyboardInterrupt: 

In [ ]:
print(i)
print(num_models)

In [105]:
num_models / i

1.9822294022617124

In [101]:
session.close()

In [106]:
len(all_datadirs)

1640

In [94]:
collect_data

[[<models.jp_autoreview.PipelineTest at 0x7f2b5b7bb860>],